In [18]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data')

In [19]:
data = pd.read_csv('kangwon.csv', encoding='utf-8')
data.shape

(1230355, 13)

In [20]:
# 필요없는 칼럼 삭제
del data['Unnamed: 0']
data.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,Datetime
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019-01
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019-01
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019-01
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019-01
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019-01


In [21]:
# 결측치 확인
data.isnull().sum()

CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
Datetime        0
dtype: int64

In [22]:
# 시군구도 필요없으니 시군구 변수도 삭제
del data['CARD_CCG_NM']

In [23]:
# 칼럼명 변경
cols = ['가맹점주소', '가맹점업종', '사용자거주시도', '사용자거주구군', '연령대',
       '성별', '가구생애주기', '카드사용인원수(환불사용자포함)', '카드사용금액', '카드사용건수(환불사용자제외)',
       '날짜']
data.columns = cols

In [24]:
# 카드사용금액을 맨 마지막 칼럼 순서로 돌리기
col_list = data.columns.tolist()
col_list.remove('날짜')
col_list.insert(0,'날짜')
col_list.remove('카드사용금액')
col_list.insert(11,'카드사용금액')

In [25]:
data = data[col_list]
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,건강보조식품 소매업,강원,강릉시,20s,1,1,4,4,311200
1,2019-01,강원,건강보조식품 소매업,강원,강릉시,30s,1,2,7,8,1374500
2,2019-01,강원,건강보조식품 소매업,강원,강릉시,30s,2,2,6,6,818700
3,2019-01,강원,건강보조식품 소매업,강원,강릉시,40s,1,3,4,5,1717000
4,2019-01,강원,건강보조식품 소매업,강원,강릉시,40s,1,4,3,3,1047300


In [9]:
data.describe()

,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
count,1.230355e+06,1.230355e+06,1.230355e+06,1.230355e+06,1.230355e+06
mean,1.379023e+00,2.852438e+00,2.879178e+01,5.232842e+01,1.291994e+06
std,4.851441e-01,1.354864e+00,2.037824e+02,4.719245e+02,1.047847e+07
min,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,1.820000e+03
25%,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,7.930000e+04
50%,1.000000e+00,3.000000e+00,6.000000e+00,8.000000e+00,1.965000e+05
75%,2.000000e+00,4.000000e+00,1.400000e+01,2.000000e+01,5.211000e+05
max,2.000000e+00,5.000000e+00,1.396900e+04,4.875400e+04,8.586119e+08


In [10]:
# 업종 종류 개수 살펴보기
data['가맹점업종'].nunique()

40

In [11]:
# 업종별로 카드사용건수 합계 집계해보기
upjong_cnt = data.groupby('가맹점업종').agg({'카드사용건수(환불사용자제외)':'sum'}).rename(columns={'카드사용건수(환불사용자제외)':'카드사용건수합계'})
upjong_cnt.head()

,카드사용건수합계
가맹점업종,
건강보조식품 소매업,16170
골프장 운영업,465266
과실 및 채소 소매업,428427
관광 민예품 및 선물용품 소매업,22012
그외 기타 분류안된 오락관련 서비스업,10


In [12]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [13]:
upjong_cnt = upjong_cnt.sort_values(by='카드사용건수합계', ascending=False)
upjong_cnt.iplot(kind='bar', theme='white')

In [14]:
# 업종에 따른 카드사용금액 평균값으로 비교해보기
upjong_cnt = data.groupby('가맹점업종').agg({'카드사용금액':'mean'}).rename(columns={'카드사용금액':'카드사용금액_평균'})
upjong_cnt = upjong_cnt.sort_values(by='카드사용금액_평균', ascending=False)
upjong_cnt.iplot(kind='bar', theme='white')

In [88]:
# 카드사용금액 평균값이랑 카드사용건수 합계 두 개 기준으로 업종에 따라 오름차순으로 정렬
# 오름차순으로 정렬하는 이유는 추후에 업종별 순위로 Weight를 편하게 매기기 위함임
upjong  = data.groupby('가맹점업종').agg({'카드사용건수(환불사용자제외)':'sum','카드사용금액':'mean'}).rename(columns={'카드사용건수(환불사용자제외)':'카드사용건수합계',
                                                                                               '카드사용금액':'카드사용금액_평균'})
# sort_values 기준을 다량으로 부여할 때, 앞의 요소를 우선순위로 sort한다.
upjong = upjong.sort_values(by=['카드사용금액_평균','카드사용건수합계'], ascending=True)
upjong.head(20)

,카드사용건수합계,카드사용금액_평균
가맹점업종,,
택시 운송업,206,62926.428571
버스 운송업,8018,63660.787671
내항 여객 운송업,1571,87656.054187
그외 기타 분류안된 오락관련 서비스업,10,88500.000000
관광 민예품 및 선물용품 소매업,22012,103254.744354
여행사업,69465,148423.670231
기타 수상오락 서비스업,3698,176712.009804
비알콜 음료점업,2171921,288395.523733
면세점,88,344198.571429


In [89]:
pd.options.display.max_rows=41
upjong_ranking = upjong.index.tolist()
print(upjong_ranking)

['택시 운송업', '버스 운송업', '내항 여객 운송업', '그외 기타 분류안된 오락관련 서비스업', '관광 민예품 및 선물용품 소매업', '여행사업', '기타 수상오락 서비스업', '비알콜 음료점업', '면세점', '피자 햄버거 샌드위치 및 유사 음식점업', '전시 및 행사 대행업', '그외 기타 스포츠시설 운영업', '수산물 소매업', '빵 및 과자류 소매업', '욕탕업', '과실 및 채소 소매업', '여관업', '서양식 음식점업', '휴양콘도 운영업', '체인화 편의점', '자동차 임대업', '호텔업', '마사지업', '기타 외국식 음식점업', '중식 음식점업', '일식 음식점업', '차량용 가스 충전업', '건강보조식품 소매업', '슈퍼마켓', '화장품 및 방향제 소매업', '골프장 운영업', '한식 음식점업', '기타 대형 종합 소매업', '육류 소매업', '기타 주점업', '차량용 주유소 운영업', '기타음식료품위주종합소매업', '스포츠 및 레크레이션 용품 임대업', '그외 기타 종합 소매업', '일반유흥 주점업']


In [90]:
# dict형태로 만들어주기 위해서 업종별 옆에다가 가중치 점수 추가해주기
for weight in range(0, 40):
    upjong_ranking.insert(2*weight+1, weight+1)
print(upjong_ranking)

['택시 운송업', 1, '버스 운송업', 2, '내항 여객 운송업', 3, '그외 기타 분류안된 오락관련 서비스업', 4, '관광 민예품 및 선물용품 소매업', 5, '여행사업', 6, '기타 수상오락 서비스업', 7, '비알콜 음료점업', 8, '면세점', 9, '피자 햄버거 샌드위치 및 유사 음식점업', 10, '전시 및 행사 대행업', 11, '그외 기타 스포츠시설 운영업', 12, '수산물 소매업', 13, '빵 및 과자류 소매업', 14, '욕탕업', 15, '과실 및 채소 소매업', 16, '여관업', 17, '서양식 음식점업', 18, '휴양콘도 운영업', 19, '체인화 편의점', 20, '자동차 임대업', 21, '호텔업', 22, '마사지업', 23, '기타 외국식 음식점업', 24, '중식 음식점업', 25, '일식 음식점업', 26, '차량용 가스 충전업', 27, '건강보조식품 소매업', 28, '슈퍼마켓', 29, '화장품 및 방향제 소매업', 30, '골프장 운영업', 31, '한식 음식점업', 32, '기타 대형 종합 소매업', 33, '육류 소매업', 34, '기타 주점업', 35, '차량용 주유소 운영업', 36, '기타음식료품위주종합소매업', 37, '스포츠 및 레크레이션 용품 임대업', 38, '그외 기타 종합 소매업', 39, '일반유흥 주점업', 40]


In [91]:
# dict 형태로 변환해주는 함수 정의
def convert_to_dict(lst):
    dict_type = {lst[i] : lst[i+1] for i in range(0, len(lst), 2)}
    return dict_type

In [92]:
upjong_weight = convert_to_dict(upjong_ranking)
print(upjong_weight)

{'택시 운송업': 1, '버스 운송업': 2, '내항 여객 운송업': 3, '그외 기타 분류안된 오락관련 서비스업': 4, '관광 민예품 및 선물용품 소매업': 5, '여행사업': 6, '기타 수상오락 서비스업': 7, '비알콜 음료점업': 8, '면세점': 9, '피자 햄버거 샌드위치 및 유사 음식점업': 10, '전시 및 행사 대행업': 11, '그외 기타 스포츠시설 운영업': 12, '수산물 소매업': 13, '빵 및 과자류 소매업': 14, '욕탕업': 15, '과실 및 채소 소매업': 16, '여관업': 17, '서양식 음식점업': 18, '휴양콘도 운영업': 19, '체인화 편의점': 20, '자동차 임대업': 21, '호텔업': 22, '마사지업': 23, '기타 외국식 음식점업': 24, '중식 음식점업': 25, '일식 음식점업': 26, '차량용 가스 충전업': 27, '건강보조식품 소매업': 28, '슈퍼마켓': 29, '화장품 및 방향제 소매업': 30, '골프장 운영업': 31, '한식 음식점업': 32, '기타 대형 종합 소매업': 33, '육류 소매업': 34, '기타 주점업': 35, '차량용 주유소 운영업': 36, '기타음식료품위주종합소매업': 37, '스포츠 및 레크레이션 용품 임대업': 38, '그외 기타 종합 소매업': 39, '일반유흥 주점업': 40}


---

In [33]:
# 가중치 부여해주는 함수 되는지 test용
test = pd.DataFrame(
{'가맹점업종':['택시 운송업', '버스 운송업', '내항 여객 운송업'],
'지역':['강원','강원','강원']},
columns=['지역','가맹점업종'])
test

,지역,가맹점업종
0,강원,택시 운송업
1,강원,버스 운송업
2,강원,내항 여객 운송업


In [36]:
test.apply(give_to_weight, axis=1)

,지역,가맹점업종
0,강원,1
1,강원,2
2,강원,3


---

In [95]:
# upjong_weight를 이용해 원본 데이터에다가 업종별로 정해진 가중치 부여해주기
def give_to_weight(df):
    if df['가맹점업종'] in upjong_weight.keys():
        df['가맹점업종'] = upjong_weight[df['가맹점업종']]
    return df

In [96]:
data = data.apply(give_to_weight, axis=1)
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,28,강원,강릉시,20s,1,1,4,4,311200
1,2019-01,강원,28,강원,강릉시,30s,1,2,7,8,1374500
2,2019-01,강원,28,강원,강릉시,30s,2,2,6,6,818700
3,2019-01,강원,28,강원,강릉시,40s,1,3,4,5,1717000
4,2019-01,강원,28,강원,강릉시,40s,1,4,3,3,1047300


In [40]:
data.to_csv('./categorical_encoding.csv', encoding='utf-8')

---
- 이제 업종 말고 다른 범주형 변수 인코딩시키기

In [3]:
data = pd.read_csv('categorical_encoding.csv', encoding='utf-8')
data.shape

(1230355, 12)

In [5]:
del data['Unnamed: 0']
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,28,강원,강릉시,20s,1,1,4,4,311200
1,2019-01,강원,28,강원,강릉시,30s,1,2,7,8,1374500
2,2019-01,강원,28,강원,강릉시,30s,2,2,6,6,818700
3,2019-01,강원,28,강원,강릉시,40s,1,3,4,5,1717000
4,2019-01,강원,28,강원,강릉시,40s,1,4,3,3,1047300


In [6]:
# 연령대별로 카드사용건수 얼마나 많은지 살펴보기
age_cnt = data.groupby('연령대').agg({'카드사용건수(환불사용자제외)':'sum'}).rename(columns={'카드사용건수(환불사용자제외)':'카드사용건수합계'})
age_cnt = age_cnt.sort_values(by=['카드사용건수합계'], ascending=False)
age_cnt.head()

,카드사용건수합계
연령대,
40s,15380124
50s,14859205
20s,12974401
30s,11558837
60s,7685819


In [8]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [9]:
age_cnt.iplot(kind='bar', theme='white')

In [10]:
age_money = data.groupby('연령대').agg({'카드사용금액':'mean'}).rename(columns={'카드사용금액':'카드사용금액_평균'})
age_money = age_money.sort_values(by=['카드사용금액_평균'], ascending=False)
age_money.head()

,카드사용금액_평균
연령대,
60s,2.059677e+06
70s,2.058285e+06
50s,1.856255e+06
40s,1.289848e+06
30s,9.460089e+05


In [11]:
age_money.iplot(kind='bar', theme='white')
# 카드사용건수에서 60대는 낮은 수치를 보였지만 
# 카드사용금액에서는 가장 높은 금액대를 카드로 사용함
# 이는 결국 50~70대 연령대가 한 번을 결제하더라도 많은 금액을 결제하는 것을 알 수 있음

In [17]:
# 연령대에 따라 카드사용금액 평균과 카드사용건수 합계 집계하기
# 정렬할 때는 카드사용금액 평균을 우선 기준으로 내림차순으로 정렬
age_cnt_money = data.groupby('연령대').agg({'카드사용금액':'mean',
                        '카드사용건수(환불사용자제외)':'sum'}).rename(columns={'카드사용금액':'카드사용금액_평균',
                                                                 '카드사용건수(환불사용자제외)':'카드사용건수합계'})
age_cnt_money = age_cnt_money.sort_values(by=['카드사용금액_평균', '카드사용건수합계'], ascending=False)
age_cnt_money

,카드사용금액_평균,카드사용건수합계
연령대,,
60s,2.059677e+06,7685819
70s,2.058285e+06,1570646
50s,1.856255e+06,14859205
40s,1.289848e+06,15380124
30s,9.460089e+05,11558837
20s,6.012183e+05,12974401
10s,3.186492e+05,353497


In [18]:
# 70대가 왜이렇게 카드사용건수합계가 엄청 적음에도 불구하고 카드 사용금액 평균값이 2위다!!
# 대체 70대가 자주 소비하는 업종은 무엇일까?
age_cnt_money.iplot(kind='bar', theme='white')

In [19]:
age_70 = data[data['연령대'] == '70s']
age_70.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
10,2019-01,강원,28,강원,강릉시,70s,1,5,7,7,660000
11,2019-01,강원,28,강원,강릉시,70s,2,5,7,8,1554750
20,2019-01,강원,31,강원,강릉시,70s,1,5,10,19,2130300
47,2019-01,강원,16,강원,강릉시,70s,1,5,84,103,5273600
48,2019-01,강원,16,강원,강릉시,70s,2,5,93,114,5414000


In [23]:
age_70['가맹점업종'] = age_70['가맹점업종'].astype(str)
age_70.dtypes

날짜                  object
가맹점주소               object
가맹점업종               object
사용자거주시도             object
사용자거주구군             object
연령대                 object
성별                   int64
가구생애주기               int64
카드사용인원수(환불사용자포함)     int64
카드사용건수(환불사용자제외)      int64
카드사용금액               int64
dtype: object

In [25]:
age_70.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
10,2019-01,강원,28,강원,강릉시,70s,1,5,7,7,660000
11,2019-01,강원,28,강원,강릉시,70s,2,5,7,8,1554750
20,2019-01,강원,31,강원,강릉시,70s,1,5,10,19,2130300
47,2019-01,강원,16,강원,강릉시,70s,1,5,84,103,5273600
48,2019-01,강원,16,강원,강릉시,70s,2,5,93,114,5414000


In [27]:
age_70['가맹점업종'].value_counts()

32    6524
36    3117
29    2780
20    2219
37    1554
31    1477
16    1334
33    1025
13    1001
19     855
34     838
14     607
26     589
25     514
38     513
18     486
8      469
27     441
24     428
10     400
30     369
22     367
15     287
17     195
28     158
6      133
11     122
40     102
39     100
23      59
35      53
5       36
2       17
3       13
12       4
Name: 가맹점업종, dtype: int64

- 70대 연령대가 가장 많이 카드를 사용하는 업종
    * 기타 대형 종합 소매업
    * 기타음식료품위주종합소매업
    * 화장품 및 방향제 소매업
    * 자동차 입대업

In [30]:
# 모든 연령대의 카드사용금액 합계를 구해서 각 연령대별 사용금액 합계의 비율 구해주어서 가중치 만들어주기
age_money = data.groupby('연령대').agg({'카드사용금액':'sum'}).rename(columns={'카드사용금액':'카드사용금액_합계'})
age_money = age_money.sort_values(by=['카드사용금액_합계'], ascending=False)
age_money.head()

,카드사용금액_합계
연령대,
50s,464627966950
40s,393332733020
60s,280701047795
30s,237249571254
20s,151375950129


In [32]:
total_money = age_money['카드사용금액_합계'].sum()
total_money

1589611299919

In [33]:
def divided_by_total(row):
    row['카드사용금액_비율'] = row['카드사용금액_합계'] / total_money
    return row

In [35]:
age_money_ratio = age_money.apply(divided_by_total, axis=1)
age_money_ratio

,카드사용금액_합계,카드사용금액_비율
연령대,,
50s,4.646280e+11,0.292290
40s,3.933327e+11,0.247440
60s,2.807010e+11,0.176585
30s,2.372496e+11,0.149250
20s,1.513760e+11,0.095228
70s,6.007309e+10,0.037791
10s,2.250938e+09,0.001416


In [36]:
age_money_ratio = age_money_ratio.reset_index()
age_money_ratio = age_money_ratio[['연령대', '카드사용금액_비율']]
age_money_ratio.head()

,연령대,카드사용금액_비율
0,50s,0.292290
1,40s,0.247440
2,60s,0.176585
3,30s,0.149250
4,20s,0.095228


In [37]:
# convert each row of dataframe to dictionary type(to_dict() function)
age_money_ratio = age_money_ratio.set_index('연령대')
age_money_ratio_dict = age_money_ratio.to_dict()
age_money_ratio_dict

{'카드사용금액_비율': {'50s': 0.29229030202142847,
  '40s': 0.24743956779877105,
  '60s': 0.1765847083556234,
  '30s': 0.14925005330931485,
  '20s': 0.09522828010640934,
  '70s': 0.037791057829710366,
  '10s': 0.0014160305787425508}}

In [44]:
age_money_ratio_dict = age_money_ratio_dict['카드사용금액_비율']
age_money_ratio_dict

{'50s': 0.29229030202142847,
 '40s': 0.24743956779877105,
 '60s': 0.1765847083556234,
 '30s': 0.14925005330931485,
 '20s': 0.09522828010640934,
 '70s': 0.037791057829710366,
 '10s': 0.0014160305787425508}

In [60]:
for k, v in age_money_ratio_dict.items():
    age_money_ratio_dict[k] = round(v, 5)

In [61]:
age_money_ratio_dict

{'50s': 0.29229,
 '40s': 0.24744,
 '60s': 0.17658,
 '30s': 0.14925,
 '20s': 0.09523,
 '70s': 0.03779,
 '10s': 0.00142}

In [62]:
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,28,강원,강릉시,20s,1,1,4,4,311200
1,2019-01,강원,28,강원,강릉시,30s,1,2,7,8,1374500
2,2019-01,강원,28,강원,강릉시,30s,2,2,6,6,818700
3,2019-01,강원,28,강원,강릉시,40s,1,3,4,5,1717000
4,2019-01,강원,28,강원,강릉시,40s,1,4,3,3,1047300


In [63]:
# 연령대 범주형 변수를 위에서 정해준 가중치에 맞게 숫자값으로 encoding하기
def age_encoding(row):
    if row['연령대'] in age_money_ratio_dict:
        row['연령대'] = age_money_ratio_dict[row['연령대']]
    return row

In [64]:
encoding_df = data.apply(age_encoding, axis=1)
encoding_df.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,28,강원,강릉시,0.09523,1,1,4,4,311200
1,2019-01,강원,28,강원,강릉시,0.14925,1,2,7,8,1374500
2,2019-01,강원,28,강원,강릉시,0.14925,2,2,6,6,818700
3,2019-01,강원,28,강원,강릉시,0.24744,1,3,4,5,1717000
4,2019-01,강원,28,강원,강릉시,0.24744,1,4,3,3,1047300


In [71]:
# 성별에 대해서는 원-핫 인코딩 적용하기
sex_dummies = pd.get_dummies(encoding_df['성별'], prefix='성별')
sex_dummies.head()

,성별_1,성별_2
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [77]:
# 원핫인코딩으로 생겨난 파생변수 칼럼 이름 바꿔주기
sex_dummies.columns = ['남자', '여자']
# 원본 데이터와 합쳐주고 성별 칼럼 없애주기
encoding_df2 = pd.concat([encoding_df, sex_dummies], axis=1)
del encoding_df2['성별']
encoding_df2.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액,남자,여자
0,2019-01,강원,28,강원,강릉시,0.09523,1,4,4,311200,1,0
1,2019-01,강원,28,강원,강릉시,0.14925,2,7,8,1374500,1,0
2,2019-01,강원,28,강원,강릉시,0.14925,2,6,6,818700,0,1
3,2019-01,강원,28,강원,강릉시,0.24744,3,4,5,1717000,1,0
4,2019-01,강원,28,강원,강릉시,0.24744,4,3,3,1047300,1,0


In [78]:
cols = encoding_df2.columns.tolist()
cols.remove('남자')
cols.insert(6, '남자')
cols.remove('여자')
cols.insert(7,'여자')
encoding_df2 = encoding_df2[cols]
encoding_df2.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,28,강원,강릉시,0.09523,1,0,1,4,4,311200
1,2019-01,강원,28,강원,강릉시,0.14925,1,0,2,7,8,1374500
2,2019-01,강원,28,강원,강릉시,0.14925,0,1,2,6,6,818700
3,2019-01,강원,28,강원,강릉시,0.24744,1,0,3,4,5,1717000
4,2019-01,강원,28,강원,강릉시,0.24744,1,0,4,3,3,1047300


In [91]:
# 가구생애주기 인코딩하기
# 가구생애주기에 따른 카드사용건수, 카드사용금액 EDA
life_cycle = encoding_df2[['가구생애주기', '카드사용건수(환불사용자제외)', '카드사용금액']]
life_cycle['가구생애주기'] = life_cycle['가구생애주기'].astype(str)

In [92]:
life_cycle = life_cycle.groupby('가구생애주기').agg({'카드사용금액':'sum',
                                 '카드사용건수(환불사용자제외)':'sum'}).rename(columns={'카드사용금액':'카드사용금액_합계',
                                                                          '카드사용건수(환불사용자제외)':'카드사용건수합계'})
life_cycle = life_cycle.sort_values(by=['카드사용금액_합계', '카드사용건수합계'], ascending=False)
life_cycle.head()

,카드사용금액_합계,카드사용건수합계
가구생애주기,,
4,552756848061,18198602
5,340774140357,9256465
3,274692520047,10733454
2,268632995787,13070237
1,152754795667,13123771


In [93]:
life_cycle.iplot(kind='bar', theme='white')

In [101]:
life_total = life_cycle[['카드사용금액_합계']]
life_total.head(1)

,카드사용금액_합계
가구생애주기,
4,552756848061


In [97]:
total_money = life_total['카드사용금액_합계'].sum()
total_money

1589611299919

In [98]:
def divided_by_total(row):
    row['카드사용금액_비율'] = row['카드사용금액_합계'] / total_money
    return row

In [102]:
life_total = life_total.apply(divided_by_total, axis=1)
del life_total['카드사용금액_합계']
life_total

,카드사용금액_비율
가구생애주기,
4,0.347731
5,0.214376
3,0.172805
2,0.168993
1,0.096096


In [103]:
life_total_dict = life_total.to_dict()
life_total_dict

{'카드사용금액_비율': {'4': 0.34773082456646237,
  '5': 0.21437576618533377,
  '3': 0.17280483603821714,
  '2': 0.16899288260009754,
  '1': 0.09609569060988919}}

In [104]:
life_total_ratio_dict = life_total_dict['카드사용금액_비율']
life_total_ratio_dict

{'4': 0.34773082456646237,
 '5': 0.21437576618533377,
 '3': 0.17280483603821714,
 '2': 0.16899288260009754,
 '1': 0.09609569060988919}

In [106]:
for k, v in life_total_ratio_dict.items():
    life_total_ratio_dict[k] = round(v, 5)
life_total_ratio_dict

{'4': 0.34773, '5': 0.21438, '3': 0.1728, '2': 0.16899, '1': 0.0961}

In [107]:
# 가구생애주기 정해진 비율 가중치에 부여하기 위한 함수
def lifecycle_encoding(row):
    if row['가구생애주기'] in life_total_ratio_dict:
        row['가구생애주기'] = life_total_ratio_dict[row['가구생애주기']]
    return row

In [110]:
# 원본 데이터의 가구생애주기 str형태로 바꿔주기
encoding_df2['가구생애주기'] = encoding_df2['가구생애주기'].astype(str)
encoding_df2.dtypes

날짜                   object
가맹점주소                object
가맹점업종                 int64
사용자거주시도              object
사용자거주구군              object
연령대                 float64
남자                    uint8
여자                    uint8
가구생애주기               object
카드사용인원수(환불사용자포함)      int64
카드사용건수(환불사용자제외)       int64
카드사용금액                int64
dtype: object

In [111]:
encoding_df3 = encoding_df2.apply(lifecycle_encoding, axis=1)
encoding_df3.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,28,강원,강릉시,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,28,강원,강릉시,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,28,강원,강릉시,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,28,강원,강릉시,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,28,강원,강릉시,0.24744,1,0,0.34773,3,3,1047300


In [113]:
encoding_df3.to_csv('./categorical_encoding.csv', encoding='utf-8')

---

In [3]:
data = pd.read_csv('categorical_encoding.csv', encoding='utf-8')
data.shape

(1230355, 13)

In [5]:
del data['Unnamed: 0']

In [6]:
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,28,강원,강릉시,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,28,강원,강릉시,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,28,강원,강릉시,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,28,강원,강릉시,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,28,강원,강릉시,0.24744,1,0,0.34773,3,3,1047300


In [97]:
data['가맹점업종'] = data['가맹점업종'].astype(str)
data.dtypes

날짜                  object
가맹점주소               object
가맹점업종               object
사용자거주시도             object
사용자거주구군             object
연령대                 object
성별                   int64
가구생애주기               int64
카드사용인원수(환불사용자포함)     int64
카드사용건수(환불사용자제외)      int64
카드사용금액               int64
dtype: object

In [98]:
upjong_sum = data.groupby('가맹점업종').agg({'카드사용금액':'sum'})
upjong_sum

,카드사용금액
가맹점업종,
1,2642910
10,22895549505
11,11883341158
12,95984620
13,9718281737
14,17362423753
15,3235384981
16,16675489268
17,10594672811


In [99]:
upjong_sum = upjong_sum.rename(columns={'카드사용금액':'카드사용금액_합계'})
upjong_sum = upjong_sum.sort_values(by=['카드사용금액_합계'], ascending=False)
upjong_sum.head()

,카드사용금액_합계
가맹점업종,
32,370698901284
36,257288862505
29,152742321569
20,144823676480
33,115954856819


In [100]:
total_sum = upjong_sum['카드사용금액_합계'].sum()
total_sum

1589611299919

In [101]:
# 업종별 합계금액을 총금액 사용에 대한 비율 계산하는 함수 정의
def divided_by_total(row):
    row['카드사용금액_비율'] = row['카드사용금액_합계'] / total_sum
    return row

In [102]:
upjong_sum = upjong_sum.apply(divided_by_total, axis=1)
del upjong_sum['카드사용금액_합계']
upjong_sum.head()

,카드사용금액_비율
가맹점업종,
32,0.233201
36,0.161856
29,0.096088
20,0.091106
33,0.072945


In [103]:
# 업종별 비율 dictionary 형태로 전환
upjong_ratio = upjong_sum.to_dict()
upjong_ratio = upjong_ratio['카드사용금액_비율']
print(upjong_ratio)

{'32': 0.23320097265469192, '36': 0.16185646297186637, '29': 0.09608784334685033, '20': 0.0911063456125278, '33': 0.07294541553957787, '37': 0.07099713527498877, '31': 0.03226630758136506, '38': 0.026928957946625847, '19': 0.022558520065771574, '18': 0.020850329138757933, '26': 0.017910108991078963, '34': 0.017115228477166924, '10': 0.014403237763953152, '25': 0.013293838449737241, '40': 0.012295878907627272, '8': 0.011409285591970913, '14': 0.010922433524399781, '16': 0.010490293613822269, '24': 0.010262673458493455, '22': 0.00938328400519048, '11': 0.007475627003032456, '17': 0.006664945582319314, '13': 0.006113621447894339, '30': 0.0056989849087393995, '27': 0.00527748276665338, '39': 0.004366606710932223, '35': 0.002984181727471187, '15': 0.002035330889485286, '28': 0.0011454525908898493, '6': 0.0011178381659029128, '23': 0.00033231840263523403, '5': 0.0002876250363993371, '12': 6.0382446957247335e-05, '2': 4.677608922620823e-05, '7': 4.53560565426742e-05, '21': 3.572378794922609e-

In [104]:
# ratio 값들 소수 일곱째자리 이후로 짤라주기
for k, v in upjong_ratio.items():
    upjong_ratio[k] = round(v, 7)
print(upjong_ratio)

{'32': 0.233201, '36': 0.1618565, '29': 0.0960878, '20': 0.0911063, '33': 0.0729454, '37': 0.0709971, '31': 0.0322663, '38': 0.026929, '19': 0.0225585, '18': 0.0208503, '26': 0.0179101, '34': 0.0171152, '10': 0.0144032, '25': 0.0132938, '40': 0.0122959, '8': 0.0114093, '14': 0.0109224, '16': 0.0104903, '24': 0.0102627, '22': 0.0093833, '11': 0.0074756, '17': 0.0066649, '13': 0.0061136, '30': 0.005699, '27': 0.0052775, '39': 0.0043666, '35': 0.0029842, '15': 0.0020353, '28': 0.0011455, '6': 0.0011178, '23': 0.0003323, '5': 0.0002876, '12': 6.04e-05, '2': 4.68e-05, '7': 4.54e-05, '21': 3.57e-05, '3': 2.24e-05, '9': 3e-06, '1': 1.7e-06, '4': 1e-07}


In [38]:
# 업종별 가중치 총금액 비율에 기반해서 다시 부여해주기
def upjong_encoding(row):
    if row['가맹점업종'] in upjong_ratio.keys():
        row['가맹점업종'] = upjong_ratio[row['가맹점업종']]
    return row

In [40]:
data2 = data.apply(upjong_encoding, axis=1)
data2.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,0.001146,강원,강릉시,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,0.001146,강원,강릉시,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,0.001146,강원,강릉시,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,0.001146,강원,강릉시,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,0.001146,강원,강릉시,0.24744,1,0,0.34773,3,3,1047300


In [49]:
# 최대한 0~1사이값으로 맞춰주기
data2['가맹점업종'] = data2['가맹점업종'] * 0.1
data2.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,사용자거주구군,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,0.000916,강원,강릉시,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,0.000916,강원,강릉시,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,0.000916,강원,강릉시,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,0.000916,강원,강릉시,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,0.000916,강원,강릉시,0.24744,1,0,0.34773,3,3,1047300


In [53]:
# 사용자 거주구군을 필요없으므로 삭제
# 사용자거주시도만 변수로 집어넣자!
del data2['사용자거주구군']
data2.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,0.000916,강원,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,0.000916,강원,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,0.000916,강원,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,0.000916,강원,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,0.000916,강원,0.24744,1,0,0.34773,3,3,1047300


In [55]:
# 강원도에서 카드사용한 횟수 합계 사용자 거주 시도에 따라 출력
data2['사용자거주시도'].value_counts()
# 가맹점 주소가 강원도임에도 불구하고 경기와 서울이 가장 많았음

경기    386081
서울    303381
강원    284197
인천     64622
충북     31221
경북     25763
대구     22947
충남     21455
부산     19389
경남     16352
대전     14913
울산      9601
전북      9213
광주      8390
전남      5424
세종      4219
제주      3187
Name: 사용자거주시도, dtype: int64

In [56]:
# 사용자거주시도에 따른 카드사용금액 얼마나 사용했는지 합계로 집계해서 살펴보기
user_sido_sum = data2.groupby('사용자거주시도').agg({'카드사용금액':'sum'}).rename(columns={'카드사용금액':'카드사용금액_합계'})
user_sido_sum = user_sido_sum.sort_values(by='카드사용금액_합계', ascending=False)
user_sido_sum

# 카드사용횟수는 비록 경기, 서울에 거주하는 사람들이 많았지만 카드사용금액으로 따지면 그래도 강원도 내에서는 강원도 주민들이 압도적으로 많이 소비했다!

,카드사용금액_합계
사용자거주시도,
강원,1204443934968
경기,162823451073
서울,152023451502
인천,23471220043
충북,11496202480
경북,6504213645
대구,5805140672
충남,5225338407
부산,3404369910


In [57]:
# iplot을 이용해서 시각화
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [58]:
user_sido_sum.iplot(kind='bar', theme='white')

In [63]:
user_sido_sum['카드사용금액_비율'] = user_sido_sum['카드사용금액_합계'] / total_sum
user_sido_sum.head()

,카드사용금액_합계,카드사용금액_비율
사용자거주시도,,
강원,1204443934968,0.757697
경기,162823451073,0.102430
서울,152023451502,0.095636
인천,23471220043,0.014765
충북,11496202480,0.007232


In [64]:
del user_sido_sum['카드사용금액_합계']

In [67]:
user_sido_sum['카드사용금액_비율'].min()

0.000421134932190097

In [68]:
user_sido_sum = user_sido_sum.to_dict()
user_sido_ratio = user_sido_sum['카드사용금액_비율']
for k, v in user_sido_ratio.items():
    user_sido_ratio[k] = round(v, 4)
print(user_sido_ratio)

{'강원': 0.7577, '경기': 0.1024, '서울': 0.0956, '인천': 0.0148, '충북': 0.0072, '경북': 0.0041, '대구': 0.0037, '충남': 0.0033, '부산': 0.0021, '대전': 0.0021, '경남': 0.002, '울산': 0.0012, '광주': 0.0011, '전북': 0.0011, '세종': 0.0007, '전남': 0.0005, '제주': 0.0004}


In [69]:
def region_encoding(row):
    if row['사용자거주시도'] in user_sido_ratio:
        row['사용자거주시도'] = user_sido_ratio[row['사용자거주시도']]
    return row

In [70]:
data3 = data2.apply(region_encoding, axis=1)
data3.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,0.000916,0.7577,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,0.000916,0.7577,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,0.000916,0.7577,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.34773,3,3,1047300


In [73]:
# 변수간 상관관계 관찰
corr_df = data3.drop(['날짜','가맹점주소'], axis=1)
corr_df.corr()

,가맹점업종,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
가맹점업종,1.000000,-0.117342,0.028312,0.026644,-0.026644,0.038040,0.035304,0.024140,0.042399
사용자거주시도,-0.117342,1.000000,-0.012189,-0.068430,0.068430,0.068088,0.160386,0.136273,0.153717
연령대,0.028312,-0.012189,1.000000,-0.016166,0.016166,0.770408,0.011802,0.004908,0.031233
남자,0.026644,-0.068430,-0.016166,1.000000,-1.000000,-0.015260,-0.003372,0.000906,0.005166
여자,-0.026644,0.068430,0.016166,-1.000000,1.000000,0.015260,0.003372,-0.000906,-0.005166
가구생애주기,0.038040,0.068088,0.770408,-0.015260,0.015260,1.000000,0.018734,0.010731,0.043451
카드사용인원수(환불사용자포함),0.035304,0.160386,0.011802,-0.003372,0.003372,0.018734,1.000000,0.941887,0.862338
카드사용건수(환불사용자제외),0.024140,0.136273,0.004908,0.000906,-0.000906,0.010731,0.941887,1.000000,0.752537
카드사용금액,0.042399,0.153717,0.031233,0.005166,-0.005166,0.043451,0.862338,0.752537,1.000000


In [76]:
# 인과관계가 존재할 수 밖에 없는 연령대-가구생애주기 와 카드사용에 관한 변수들 사이를 제외하고는 상관관계 없음!
# 독립적인 변수들간의 숨어있는 다중공산성 없는듯!
corr_df.corr().iplot(kind='heatmap', theme='white', colorscale='Reds')

---
* 여기부터 삽질의 흔적입니다.. 걸러주세요!

In [81]:
# 날짜와 업종별로 사용금액 합계 묶어주자
data3.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,0.000916,0.7577,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,0.000916,0.7577,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,0.000916,0.7577,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.34773,3,3,1047300


In [107]:
print(f"업종숫자 - 비율 dict : {upjong_ratio}")
print()
print()
print(f"업종종류 - 업종숫자 dict : {upjong_weight}")

업종숫자 - 비율 dict : {'32': 0.233201, '36': 0.1618565, '29': 0.0960878, '20': 0.0911063, '33': 0.0729454, '37': 0.0709971, '31': 0.0322663, '38': 0.026929, '19': 0.0225585, '18': 0.0208503, '26': 0.0179101, '34': 0.0171152, '10': 0.0144032, '25': 0.0132938, '40': 0.0122959, '8': 0.0114093, '14': 0.0109224, '16': 0.0104903, '24': 0.0102627, '22': 0.0093833, '11': 0.0074756, '17': 0.0066649, '13': 0.0061136, '30': 0.005699, '27': 0.0052775, '39': 0.0043666, '35': 0.0029842, '15': 0.0020353, '28': 0.0011455, '6': 0.0011178, '23': 0.0003323, '5': 0.0002876, '12': 6.04e-05, '2': 4.68e-05, '7': 4.54e-05, '21': 3.57e-05, '3': 2.24e-05, '9': 3e-06, '1': 1.7e-06, '4': 1e-07}


업종종류 - 업종숫자 dict : {'택시 운송업': 1, '버스 운송업': 2, '내항 여객 운송업': 3, '그외 기타 분류안된 오락관련 서비스업': 4, '관광 민예품 및 선물용품 소매업': 5, '여행사업': 6, '기타 수상오락 서비스업': 7, '비알콜 음료점업': 8, '면세점': 9, '피자 햄버거 샌드위치 및 유사 음식점업': 10, '전시 및 행사 대행업': 11, '그외 기타 스포츠시설 운영업': 12, '수산물 소매업': 13, '빵 및 과자류 소매업': 14, '욕탕업': 15, '과실 및 채소 소매업': 16, '여관업': 17, '서양식 음식점업': 1

In [117]:
num_to_weight = {}
for k, v in sorted(upjong_ratio.items()):
    num_to_weight[k] = v
print(num_to_weight)

{'1': 1.7e-06, '10': 0.0144032, '11': 0.0074756, '12': 6.04e-05, '13': 0.0061136, '14': 0.0109224, '15': 0.0020353, '16': 0.0104903, '17': 0.0066649, '18': 0.0208503, '19': 0.0225585, '2': 4.68e-05, '20': 0.0911063, '21': 3.57e-05, '22': 0.0093833, '23': 0.0003323, '24': 0.0102627, '25': 0.0132938, '26': 0.0179101, '27': 0.0052775, '28': 0.0011455, '29': 0.0960878, '3': 2.24e-05, '30': 0.005699, '31': 0.0322663, '32': 0.233201, '33': 0.0729454, '34': 0.0171152, '35': 0.0029842, '36': 0.1618565, '37': 0.0709971, '38': 0.026929, '39': 0.0043666, '4': 1e-07, '40': 0.0122959, '5': 0.0002876, '6': 0.0011178, '7': 4.54e-05, '8': 0.0114093, '9': 3e-06}


In [132]:
weight_list = []
for v in num_to_weight.values():
    weight_list.append(v)
print(weight_list)

[1.7e-06, 0.0144032, 0.0074756, 6.04e-05, 0.0061136, 0.0109224, 0.0020353, 0.0104903, 0.0066649, 0.0208503, 0.0225585, 4.68e-05, 0.0911063, 3.57e-05, 0.0093833, 0.0003323, 0.0102627, 0.0132938, 0.0179101, 0.0052775, 0.0011455, 0.0960878, 2.24e-05, 0.005699, 0.0322663, 0.233201, 0.0729454, 0.0171152, 0.0029842, 0.1618565, 0.0709971, 0.026929, 0.0043666, 1e-07, 0.0122959, 0.0002876, 0.0011178, 4.54e-05, 0.0114093, 3e-06]


In [131]:
upjong_list = []
for k in upjong_weight.keys():
    upjong_list.append(k)
print(upjong_list)

['택시 운송업', '버스 운송업', '내항 여객 운송업', '그외 기타 분류안된 오락관련 서비스업', '관광 민예품 및 선물용품 소매업', '여행사업', '기타 수상오락 서비스업', '비알콜 음료점업', '면세점', '피자 햄버거 샌드위치 및 유사 음식점업', '전시 및 행사 대행업', '그외 기타 스포츠시설 운영업', '수산물 소매업', '빵 및 과자류 소매업', '욕탕업', '과실 및 채소 소매업', '여관업', '서양식 음식점업', '휴양콘도 운영업', '체인화 편의점', '자동차 임대업', '호텔업', '마사지업', '기타 외국식 음식점업', '중식 음식점업', '일식 음식점업', '차량용 가스 충전업', '건강보조식품 소매업', '슈퍼마켓', '화장품 및 방향제 소매업', '골프장 운영업', '한식 음식점업', '기타 대형 종합 소매업', '육류 소매업', '기타 주점업', '차량용 주유소 운영업', '기타음식료품위주종합소매업', '스포츠 및 레크레이션 용품 임대업', '그외 기타 종합 소매업', '일반유흥 주점업']


In [134]:
upjong_weight_dict = dict(zip(upjong_list, weight_list))
print(upjong_weight_dict)

{'택시 운송업': 1.7e-06, '버스 운송업': 0.0144032, '내항 여객 운송업': 0.0074756, '그외 기타 분류안된 오락관련 서비스업': 6.04e-05, '관광 민예품 및 선물용품 소매업': 0.0061136, '여행사업': 0.0109224, '기타 수상오락 서비스업': 0.0020353, '비알콜 음료점업': 0.0104903, '면세점': 0.0066649, '피자 햄버거 샌드위치 및 유사 음식점업': 0.0208503, '전시 및 행사 대행업': 0.0225585, '그외 기타 스포츠시설 운영업': 4.68e-05, '수산물 소매업': 0.0911063, '빵 및 과자류 소매업': 3.57e-05, '욕탕업': 0.0093833, '과실 및 채소 소매업': 0.0003323, '여관업': 0.0102627, '서양식 음식점업': 0.0132938, '휴양콘도 운영업': 0.0179101, '체인화 편의점': 0.0052775, '자동차 임대업': 0.0011455, '호텔업': 0.0960878, '마사지업': 2.24e-05, '기타 외국식 음식점업': 0.005699, '중식 음식점업': 0.0322663, '일식 음식점업': 0.233201, '차량용 가스 충전업': 0.0729454, '건강보조식품 소매업': 0.0171152, '슈퍼마켓': 0.0029842, '화장품 및 방향제 소매업': 0.1618565, '골프장 운영업': 0.0709971, '한식 음식점업': 0.026929, '기타 대형 종합 소매업': 0.0043666, '육류 소매업': 1e-07, '기타 주점업': 0.0122959, '차량용 주유소 운영업': 0.0002876, '기타음식료품위주종합소매업': 0.0011178, '스포츠 및 레크레이션 용품 임대업': 4.54e-05, '그외 기타 종합 소매업': 0.0114093, '일반유흥 주점업': 3e-06}


In [136]:
data3.to_csv('./final_data.csv', encoding='utf-8')

---

In [3]:
data = pd.read_csv('final_data.csv', encoding='utf-8')
data.shape

(1230355, 12)

In [5]:
# 메모리과부하 때문에 위 변수 텍스트파일로 저장한 다음 복붙함!
upjong_weight_dict = {'택시 운송업': 1.7e-06, '버스 운송업': 0.0144032, '내항 여객 운송업': 0.0074756, '그외 기타 분류안된 오락관련 서비스업': 6.04e-05, '관광 민예품 및 선물용품 소매업': 0.0061136, '여행사업': 0.0109224, '기타 수상오락 서비스업': 0.0020353, '비알콜 음료점업': 0.0104903, '면세점': 0.0066649, '피자 햄버거 샌드위치 및 유사 음식점업': 0.0208503, '전시 및 행사 대행업': 0.0225585, '그외 기타 스포츠시설 운영업': 4.68e-05, '수산물 소매업': 0.0911063, '빵 및 과자류 소매업': 3.57e-05, '욕탕업': 0.0093833, '과실 및 채소 소매업': 0.0003323, '여관업': 0.0102627, '서양식 음식점업': 0.0132938, '휴양콘도 운영업': 0.0179101, '체인화 편의점': 0.0052775, '자동차 임대업': 0.0011455, '호텔업': 0.0960878, '마사지업': 2.24e-05, '기타 외국식 음식점업': 0.005699, '중식 음식점업': 0.0322663, '일식 음식점업': 0.233201, '차량용 가스 충전업': 0.0729454, '건강보조식품 소매업': 0.0171152, '슈퍼마켓': 0.0029842, '화장품 및 방향제 소매업': 0.1618565, '골프장 운영업': 0.0709971, '한식 음식점업': 0.026929, '기타 대형 종합 소매업': 0.0043666, '육류 소매업': 1e-07, '기타 주점업': 0.0122959, '차량용 주유소 운영업': 0.0002876, '기타음식료품위주종합소매업': 0.0011178, '스포츠 및 레크레이션 용품 임대업': 4.54e-05, '그외 기타 종합 소매업': 0.0114093, '일반유흥 주점업': 3e-06}
print(upjong_weight_dict)

{'택시 운송업': 1.7e-06, '버스 운송업': 0.0144032, '내항 여객 운송업': 0.0074756, '그외 기타 분류안된 오락관련 서비스업': 6.04e-05, '관광 민예품 및 선물용품 소매업': 0.0061136, '여행사업': 0.0109224, '기타 수상오락 서비스업': 0.0020353, '비알콜 음료점업': 0.0104903, '면세점': 0.0066649, '피자 햄버거 샌드위치 및 유사 음식점업': 0.0208503, '전시 및 행사 대행업': 0.0225585, '그외 기타 스포츠시설 운영업': 4.68e-05, '수산물 소매업': 0.0911063, '빵 및 과자류 소매업': 3.57e-05, '욕탕업': 0.0093833, '과실 및 채소 소매업': 0.0003323, '여관업': 0.0102627, '서양식 음식점업': 0.0132938, '휴양콘도 운영업': 0.0179101, '체인화 편의점': 0.0052775, '자동차 임대업': 0.0011455, '호텔업': 0.0960878, '마사지업': 2.24e-05, '기타 외국식 음식점업': 0.005699, '중식 음식점업': 0.0322663, '일식 음식점업': 0.233201, '차량용 가스 충전업': 0.0729454, '건강보조식품 소매업': 0.0171152, '슈퍼마켓': 0.0029842, '화장품 및 방향제 소매업': 0.1618565, '골프장 운영업': 0.0709971, '한식 음식점업': 0.026929, '기타 대형 종합 소매업': 0.0043666, '육류 소매업': 1e-07, '기타 주점업': 0.0122959, '차량용 주유소 운영업': 0.0002876, '기타음식료품위주종합소매업': 0.0011178, '스포츠 및 레크레이션 용품 임대업': 4.54e-05, '그외 기타 종합 소매업': 0.0114093, '일반유흥 주점업': 3e-06}


In [7]:
del data['Unnamed: 0']
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,0.000916,0.7577,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,0.000916,0.7577,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,0.000916,0.7577,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.34773,3,3,1047300


In [8]:
data['업종'] = ''
data.head(1)

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액,업종
0,2019-01,강원,0.000916,0.7577,0.09523,1,0,0.0961,4,4,311200,


In [11]:
# dict의 key와 value값 바꾸기
upjong_weight_dict = {y:x for x,y in upjong_weight_dict.items()}
print(upjong_weight_dict)

{1.7e-06: '택시 운송업', 0.0144032: '버스 운송업', 0.0074756: '내항 여객 운송업', 6.04e-05: '그외 기타 분류안된 오락관련 서비스업', 0.0061136: '관광 민예품 및 선물용품 소매업', 0.0109224: '여행사업', 0.0020353: '기타 수상오락 서비스업', 0.0104903: '비알콜 음료점업', 0.0066649: '면세점', 0.0208503: '피자 햄버거 샌드위치 및 유사 음식점업', 0.0225585: '전시 및 행사 대행업', 4.68e-05: '그외 기타 스포츠시설 운영업', 0.0911063: '수산물 소매업', 3.57e-05: '빵 및 과자류 소매업', 0.0093833: '욕탕업', 0.0003323: '과실 및 채소 소매업', 0.0102627: '여관업', 0.0132938: '서양식 음식점업', 0.0179101: '휴양콘도 운영업', 0.0052775: '체인화 편의점', 0.0011455: '자동차 임대업', 0.0960878: '호텔업', 2.24e-05: '마사지업', 0.005699: '기타 외국식 음식점업', 0.0322663: '중식 음식점업', 0.233201: '일식 음식점업', 0.0729454: '차량용 가스 충전업', 0.0171152: '건강보조식품 소매업', 0.0029842: '슈퍼마켓', 0.1618565: '화장품 및 방향제 소매업', 0.0709971: '골프장 운영업', 0.026929: '한식 음식점업', 0.0043666: '기타 대형 종합 소매업', 1e-07: '육류 소매업', 0.0122959: '기타 주점업', 0.0002876: '차량용 주유소 운영업', 0.0011178: '기타음식료품위주종합소매업', 4.54e-05: '스포츠 및 레크레이션 용품 임대업', 0.0114093: '그외 기타 종합 소매업', 3e-06: '일반유흥 주점업'}


In [12]:
# 가맹점업종 칼럼 추가하기
# dict의 value에 맞는 값에 매핑되는 key 업종 할당해주기
def create_upjong(row):
    if row['가맹점업종'] in upjong_weight_dict.keys():
        row['업종'] = upjong_weight_dict[row['가맹점업종']]
    return row

In [13]:
final_data = data.apply(create_upjong, axis=1)
final_data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액,업종
0,2019-01,강원,0.000916,0.7577,0.09523,1,0,0.09610,4,4,311200,
1,2019-01,강원,0.000916,0.7577,0.14925,1,0,0.16899,7,8,1374500,
2,2019-01,강원,0.000916,0.7577,0.14925,0,1,0.16899,6,6,818700,
3,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.17280,4,5,1717000,
4,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.34773,3,3,1047300,


In [28]:
del final_data['업종']

* 여기까지 삽질입니다.. 밑의 코드랑 관련이 없습니다
---

In [27]:
data = data.rename(columns={'가맹점업종':'가맹점업종_이름'})
data.head()

,날짜,가맹점주소,가맹점업종_이름,사용자거주시도,사용자거주구군,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,건강보조식품 소매업,강원,강릉시,20s,1,1,4,4,311200
1,2019-01,강원,건강보조식품 소매업,강원,강릉시,30s,1,2,7,8,1374500
2,2019-01,강원,건강보조식품 소매업,강원,강릉시,30s,2,2,6,6,818700
3,2019-01,강원,건강보조식품 소매업,강원,강릉시,40s,1,3,4,5,1717000
4,2019-01,강원,건강보조식품 소매업,강원,강릉시,40s,1,4,3,3,1047300


In [30]:
sub_data = data[['가맹점업종_이름']]
sub_data.head()

,가맹점업종_이름
0,건강보조식품 소매업
1,건강보조식품 소매업
2,건강보조식품 소매업
3,건강보조식품 소매업
4,건강보조식품 소매업


In [35]:
final_data2 = pd.concat([final_data, sub_data], axis=1)
final_data2.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액,가맹점업종_이름
0,2019-01,강원,0.000916,0.7577,0.09523,1,0,0.09610,4,4,311200,건강보조식품 소매업
1,2019-01,강원,0.000916,0.7577,0.14925,1,0,0.16899,7,8,1374500,건강보조식품 소매업
2,2019-01,강원,0.000916,0.7577,0.14925,0,1,0.16899,6,6,818700,건강보조식품 소매업
3,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.17280,4,5,1717000,건강보조식품 소매업
4,2019-01,강원,0.000916,0.7577,0.24744,1,0,0.34773,3,3,1047300,건강보조식품 소매업


In [38]:
cols = final_data2.columns.tolist()
cols.remove('가맹점업종_이름')
cols.insert(2, '가맹점업종_이름')
print(cols)

['날짜', '가맹점주소', '가맹점업종_이름', '가맹점업종', '사용자거주시도', '연령대', '남자', '여자', '가구생애주기', '카드사용인원수(환불사용자포함)', '카드사용건수(환불사용자제외)', '카드사용금액']


In [40]:
final_data2 = final_data2[cols]
final_data2 = final_data2.rename(columns={'가맹점업종':'업종_가중치',
                                         '가맹점업종_이름':'가맹점업종'})
final_data2.head()

,날짜,가맹점주소,가맹점업종,업종_가중치,사용자거주시도,연령대,남자,여자,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,2019-01,강원,건강보조식품 소매업,0.000916,0.7577,0.09523,1,0,0.09610,4,4,311200
1,2019-01,강원,건강보조식품 소매업,0.000916,0.7577,0.14925,1,0,0.16899,7,8,1374500
2,2019-01,강원,건강보조식품 소매업,0.000916,0.7577,0.14925,0,1,0.16899,6,6,818700
3,2019-01,강원,건강보조식품 소매업,0.000916,0.7577,0.24744,1,0,0.17280,4,5,1717000
4,2019-01,강원,건강보조식품 소매업,0.000916,0.7577,0.24744,1,0,0.34773,3,3,1047300


In [43]:
final_data2.to_csv('final_data.csv', encoding='utf-8')